### GROUP 19 - AI-Detection Content Modeling Phase (Hugging Face Models)
NOTE: I did my own pre-processing steps and tested two Hugging Face models using the "transformers" library here (OpenAI and ChatGPT Roberta Models). The processed datasets with Classificaton and Confidence Scores took more than an hour to classify but the results are better compared to the custom model when it comes to Reddit Post contents. I highly recommend using this instead and its also easier for backend integration -- Dee

Source AI-Human Dataset: https://www.kaggle.com/datasets/shanegerami/ai-vs-human-text

Source Reddit Dataset: https://www.kaggle.com/datasets/cosmos98/twitter-and-reddit-sentimental-analysis-dataset

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
from transformers import pipeline

In [2]:
# Load the Reddit dataset
reddit_data = pd.read_csv("Reddit_Data.csv")

# Display the first few rows
print(reddit_data.head())

                                       clean_comment  category
0   family mormon have never tried explain them t...         1
1  buddhism has very much lot compatible with chr...         1
2  seriously don say thing first all they won get...        -1
3  what you have learned yours and only yours wha...         0
4  for your own benefit you may want read living ...         1


In [3]:
# Rename 'clean_comment' column to 'text' (if it exists)
reddit_data = reddit_data.rename(columns={"clean_comment": "text"})

# Drop the 'generated' column (removes sentiment classification)
reddit_data = reddit_data.drop(columns=['category'], errors='ignore')

# Display the first few rows
print(reddit_data.head())

                                                text
0   family mormon have never tried explain them t...
1  buddhism has very much lot compatible with chr...
2  seriously don say thing first all they won get...
3  what you have learned yours and only yours wha...
4  for your own benefit you may want read living ...


In [4]:
# Check for missing values in the 'text' column
print(reddit_data['text'].isnull().sum())

100


In [5]:
# Drop rows with missing values in the 'text' column
reddit_data = reddit_data.dropna(subset=['text'])

In [6]:
# Check for missing values again
print(reddit_data['text'].isnull().sum())

0


In [7]:
# Function to clean text
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"http\S+|www\S+", "", text)  # Remove URLs
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # Remove special characters & numbers
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text

# Apply text cleaning
reddit_data['text'] = reddit_data['text'].apply(clean_text)

In [8]:
# Display the first few rows
print(reddit_data.head())

                                                text
0  family mormon have never tried explain them th...
1  buddhism has very much lot compatible with chr...
2  seriously don say thing first all they won get...
3  what you have learned yours and only yours wha...
4  for your own benefit you may want read living ...


### Hugging Face Dataset Labeling (Cleaned and Labeled Output)

In [9]:
# Load AI Text Detector from Hugging Face
ai_detector = pipeline("text-classification", model="roberta-base-openai-detector")

Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [10]:
# Function to classify text as human (0.0) or AI-generated (1.0)
def classify_text(text):
    result = ai_detector(text[:512])[0]  # Truncate long text to 512 tokens
    return 1.0 if result['label'] == 'AI-Generated' else 0.0

# Apply classification
reddit_data['classification'] = reddit_data['text'].apply(classify_text)

In [11]:
# Display first 10 rows to verify changes
print(reddit_data.head(10))

                                                text  classification
0  family mormon have never tried explain them th...             0.0
1  buddhism has very much lot compatible with chr...             0.0
2  seriously don say thing first all they won get...             0.0
3  what you have learned yours and only yours wha...             0.0
4  for your own benefit you may want read living ...             0.0
5  you should all sit down together and watch the...             0.0
6  was teens when discovered zen meditation was t...             0.0
7                            jesus was zen meets jew             0.0
8  there are two varieties christians dogmatic th...             0.0
9  dont worry about trying explain yourself just ...             0.0


In [12]:
# Save the processed dataset
reddit_data.to_csv("Reddit_Cleaned_Classified.csv", index=False)

### ACCURACY TEST (Based on Confidence Scores - Code Only)

In [ ]:
# Function to classify text and return confidence score
def classify_text_with_confidence(text):
    result = ai_detector(text[:512])[0]  # Truncate long text to 512 tokens
    label = 1.0 if result['label'] == 'AI-Generated' else 0.0
    confidence = result['score']  # Confidence score
    return label, confidence

# Apply classification with confidence scores
reddit_data[['classification', 'confidence']] = reddit_data['text'].apply(
    lambda x: pd.Series(classify_text_with_confidence(x))
)

# Display the first 10 rows with confidence scores
print(reddit_data[['text', 'classification', 'confidence']].head(10))

In [ ]:
# Save the processed dataset
reddit_data.to_csv("Reddit_Cleaned_Classified-ConfidenceScores.csv", index=False)

### Alternative Pre-trained Model to use for Classification (Code Only)

In [ ]:
alt_detector = pipeline("text-classification", model="Hello-SimpleAI/chatgpt-detector-roberta")

def alt_classify_text(text):
    result = alt_detector(text[:512])[0]
    return 1.0 if result['label'] == 'AI' else 0.0

# Compare results
reddit_data['alt_classification'] = reddit_data['text'].apply(alt_classify_text)

# Display differences
print(reddit_data[['text', 'classification', 'alt_classification']].sample(10))

### Updating the Code to Compute AI-Generated Percentage (Code Only)

In [ ]:
# Function to classify text and return AI-Generated Percentage
def classify_text_with_percentage(text):
    result = ai_detector(text[:512])[0]  # Get classification result
    ai_generated_percentage = result['score'] if result['label'] == 'AI-Generated' else (1 - result['score'])
    return ai_generated_percentage * 100  # Convert to percentage

# Apply classification with AI-Generated Percentage
reddit_data['AI_Generated_Percentage'] = reddit_data['text'].apply(classify_text_with_percentage)

# Display first 10 rows
print(reddit_data[['text', 'AI_Generated_Percentage']].head(10))